In [39]:
from keras.models import model_from_json
import matplotlib
import numpy as np
from keras import losses
from pandas import read_csv
from keras.optimizers import Adam
import dateutil
import matplotlib.pyplot as plt
from keras.models import *
from keras.layers import *
import os
from keras.callbacks import ModelCheckpoint
from pandas import Series
from pandas import DataFrame
from pandas import concat
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"



In [40]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss=losses.mean_squared_error,
              optimizer='adam')


Loaded model from disk


# DATA PREPROCESSING

In [41]:
#helper functions skip to main code

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	#print(df)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	#print (agg)
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg


In [42]:
# create a differenced series
def difference(dataset, interval=1):
	diff = list()
	for i in range(interval, len(dataset)):
		value = dataset[i] - dataset[i - interval]
		diff.append(value)
	return Series(diff)
def prepare_data(series, n_test, n_lag, n_seq): #n_test= number of test entries, n_lag number of prev entries analysed, n_seq=number of future predicitons
	# extract raw values
	raw_values = series.values
	# transform data to be stationary
	diff_series = difference(raw_values, 1)
	diff_values = diff_series.values
	diff_values = diff_values.reshape(len(diff_values), 1)
	# rescale values to -1, 1
	scaler = MinMaxScaler(feature_range=(-1, 1))
	scaled_values = scaler.fit_transform(diff_values)
	scaled_values = scaled_values.reshape(len(scaled_values), 1)
	# transform into supervised learning problem X, y
	supervised = series_to_supervised(scaled_values, n_lag, n_seq)
	supervised_values = supervised.values
	# split into train and test sets
	train, test = supervised_values[0:-n_test], supervised_values[-n_test:]
	return scaler, train, test

In [43]:
#INVERT TRANSFORMED TO ACTUAL FORM
# invert differenced forecast
def inverse_difference(last_ob, forecast):
	# invert first forecast
	inverted = list()
	inverted.append(forecast[0] + last_ob)
	# propagate difference forecast using inverted first value
	for i in range(1, len(forecast)):
		inverted.append(forecast[i] + inverted[i-1])
	return inverted

# inverse data transform on forecasts
def inverse_transform(series, forecasts, scaler, n_test):
	inverted = list()
	for i in range(len(forecasts)):
		# create array from forecast
		forecast = np.array(forecasts[i])
		forecast = forecast.reshape(1, len(forecast))
		# invert scaling
		inv_scale = scaler.inverse_transform(forecast)
		inv_scale = inv_scale[0, :]

		# invert differencing
		index = len(series) - n_test + i - 1
		last_ob = series.values[index][1]
		
		inv_diff = inverse_difference(last_ob, inv_scale)
		# store
		inverted.append(inv_diff)
	return inverted

In [44]:
#fetch data
def getData_nex(filename):
    data = pd.read_csv(filename)
    Close = data.close
    data = Close
    scaler, train, test=prepare_data(data,1000,7,1)
    
    xtrain = train[:, 0:7]
    ytrain = train[:,7:]
    xtest = test[:, 0:7]
    ytest = test[:,7:]
    X_train = np.reshape(xtrain, (xtrain.shape[0], xtrain.shape[1],1))
    
    X_test = np.reshape(xtest, (xtest.shape[0],xtest.shape[1],1))
    return X_train,X_test,ytrain,ytest,scaler

In [45]:
#evaluate forecasts
def evaluate_forecasts(test, forecasts, n_lag, n_seq):
	for i in range(n_seq):
		y_true = np.array([row[i] for row in test])
		y_pred = np.array([forecast[i] for forecast in forecasts])
		mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
		rmse=sqrt(mean_squared_error(y_true,y_pred))
		print('t+%d MAPE: %f' % ((i+1), mape))
		print('t+%d RMSE: %f' % ((i+1), rmse))        

In [46]:
# plot the forecasts in the context of the original dataset
def plot_forecasts(series, forecasts, n_test):
	# plot the entire dataset in blue
	pyplot.plot(series.values)
	# plot the forecasts in red
	for i in range(len(forecasts)):
		off_s = len(series) - n_test + i - 1
		off_e = off_s + len(forecasts[i]) + 1
		xaxis = [x for x in range(off_s, off_e)]
		yaxis = [series.values[off_s]] + forecasts[i]
		pyplot.plot(xaxis, yaxis, color='red')
	# show the plot
	pyplot.show()

# actual working

In [47]:
import pandas as pd
import numpy as np
data=pd.read_csv('EUR_JPY.csv')
new_arr  = data.iloc[-7:,-2:-1].values  #takes last 6 values from db
new_val = 129.032 #new value in db, hardcoded for now


In [48]:
def make_prediction(new_val):
    new_data = np.append(new_arr,new_val) #creates new array
    new_df= DataFrame(new_data)
    #DATA PREPROCESSING
    new_values = new_df.values
    diff_series = difference(new_values,1)
    diff_values = diff_series.values
    diff_values = diff_values.reshape(len(diff_values), 1)
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaled_values = scaler.fit_transform(diff_values)
    scaled_values = scaled_values.reshape(len(scaled_values), 1)
    s2s = series_to_supervised(scaled_values,6)
    supervised_values = s2s.values
#     print(supervised_values)
    supervised_values = supervised_values.reshape(1,len(supervised_values[0]),1)
    #MAKE PREDICTION USING THE LOADED MODEL
    forecasts= loaded_model.predict(supervised_values)
#     print (forecasts)
    #ADD PREDICTION TO THE LAST VALUE TO OBTAIN THE NEXT VALUE
#     print(new_data[-1])
    final_prediction = new_data[-1]+forecasts
    print("predicted drop/rise "+str(forecasts))
    print("prediction : " + str(final_prediction))

In [49]:
make_prediction(new_val)

/home/nimish/.local/lib/python3.5/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


predicted drop/rise [[-0.10624623]]
prediction : [[128.92575]]
